## Introduction to Massive Data Analysis 
### HW2 - Page Rank
106070038 杜葳葳

匯入 SparkConf、SparkContext、Pyspark模組 
設定環境變數（解決版本問題）

In [42]:
# import package
from pyspark import SparkConf,SparkContext
import pyspark

# set environment variables
import os
os.environ["PYSPARK_PYTHON"]="/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8"
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8'

In [43]:
sc.stop()

建立 SparkConf 物件 
設為 local 模式、名稱設為 "hw2"

In [44]:
conf = SparkConf().setMaster("local").setAppName("hw2")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local appName=hw2>

### 讀檔

In [45]:
lines = sc.textFile("p2p-Gnutella04.txt")
#lines = sc.textFile("spec_example.txt")

In [46]:
#lines.first
#lines.take(20)

['0\t1',
 '0\t2',
 '0\t3',
 '0\t4',
 '0\t5',
 '0\t6',
 '0\t7',
 '0\t8',
 '0\t9',
 '0\t10',
 '1\t2',
 '1\t11',
 '1\t12',
 '1\t13',
 '1\t14',
 '1\t15',
 '1\t16',
 '1\t17',
 '1\t18',
 '1\t19']

Directed graph (each unordered pair of nodes is saved once): p2p-Gnutella04.txt<br>
Directed Gnutella P2P network from August 4 2002<br>
Nodes: 10876 Edges: 39994<br>
FromNodeId	ToNodeId<br>

In [47]:
Split_RDD = lines.map(lambda x : x.split("\t")).map(lambda x: (int(x[0]),int(x[1])))

In [48]:
beta = 0.8
max_nodesID = Split_RDD.flatMap(lambda x : (int(x[0]),int(x[1]))).max()
max_nodesID
Nodes = Split_RDD.flatMap(lambda x : (x[0],x[1])).max()+1
Nodes
init_value = 1/Nodes

### Example from spec
$$
r0 = 
  \begin{bmatrix}
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}
  \end{bmatrix}
$$ <br>
$$
0.8*
 \begin{bmatrix}
   0 & 0 & \frac{1}{3} & \frac{1}{5} & \frac{1}{2} \\
   \frac{1}{2} & 0 & 0 &\frac{1}{5} & 0 \\
   \frac{1}{2} & 0 & 0 &\frac{1}{5} & 0 \\
   0 & 1 & \frac{1}{3} & \frac{1}{5} & \frac{1}{2} \\
   0 & 0 & \frac{1}{3} & \frac{1}{5} & 0
  \end{bmatrix}
  \begin{bmatrix}
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}
  \end{bmatrix}
+0.2*
  \begin{bmatrix}
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}\\
  \frac{1}{5}
  \end{bmatrix}
=
  \begin{bmatrix}
  0.205333 \\
  0.152 \\
  0.152 \\
  0.365333 \\
  0.125333 
  \end{bmatrix}
=r1
$$

### 計算M矩陣（每次iterate的M皆相同）

out_nodes_sum: 計算每個node向外指向幾個不同的nodes <br>


In [49]:
out_nodes_sum = Split_RDD.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)

把沒有向外指的node補0<br>
* subtractByKey(self, other, numPartitions=None)<br>
  Return each(key, value)pair in self that has no pair with matching key in other.
* union(self, other): 兩個RDD合併<br>
  Return the union of this RDD and another one.
* collectAsMap(self)<br>
  Return the key-value pairs in this RDD to the master as a dictionary.

In [50]:
N = sc.parallelize(range(Nodes)).map(lambda x: (x, 0))
dj = N.subtractByKey(out_nodes_sum).union(out_nodes_sum)
djMap = dj.collectAsMap()
bc_djMap = sc.broadcast(djMap)

value設成$\frac{1}{d}$

In [51]:
M = Split_RDD.map(lambda x: ([x[1],x[0],1/bc_djMap.value[x[0]]]))

In [12]:
#M

[[1, 0, 0.1],
 [2, 0, 0.1],
 [3, 0, 0.1],
 [4, 0, 0.1],
 [5, 0, 0.1],
 [6, 0, 0.1],
 [7, 0, 0.1],
 [8, 0, 0.1],
 [9, 0, 0.1],
 [10, 0, 0.1],
 [2, 1, 0.1],
 [11, 1, 0.1],
 [12, 1, 0.1],
 [13, 1, 0.1],
 [14, 1, 0.1],
 [15, 1, 0.1],
 [16, 1, 0.1],
 [17, 1, 0.1],
 [18, 1, 0.1],
 [19, 1, 0.1],
 [20, 3, 0.1],
 [21, 3, 0.1],
 [22, 3, 0.1],
 [23, 3, 0.1],
 [24, 3, 0.1],
 [25, 3, 0.1],
 [26, 3, 0.1],
 [27, 3, 0.1],
 [28, 3, 0.1],
 [29, 3, 0.1],
 [30, 8, 0.1],
 [31, 8, 0.1],
 [32, 8, 0.1],
 [33, 8, 0.1],
 [34, 8, 0.1],
 [35, 8, 0.1],
 [36, 8, 0.1],
 [37, 8, 0.1],
 [38, 8, 0.1],
 [39, 8, 0.1],
 [41, 10, 0.1],
 [136, 10, 0.1],
 [137, 10, 0.1],
 [138, 10, 0.1],
 [139, 10, 0.1],
 [140, 10, 0.1],
 [141, 10, 0.1],
 [142, 10, 0.1],
 [143, 10, 0.1],
 [144, 10, 0.1],
 [40, 12, 0.1],
 [41, 12, 0.1],
 [42, 12, 0.1],
 [43, 12, 0.1],
 [44, 12, 0.1],
 [45, 12, 0.1],
 [46, 12, 0.1],
 [47, 12, 0.1],
 [48, 12, 0.1],
 [49, 12, 0.1],
 [199, 14, 0.1],
 [756, 14, 0.1],
 [1116, 14, 0.1],
 [2537, 14, 0.1],
 [2787, 14,

### 對deadend 做 renormalize處理

找出所有沒有值的column<br>
然後補上$[\frac{beta}{N}]$
* distinct(): 把重複的drop掉

In [52]:
M_no_values = sc.parallelize(range(Nodes)).subtract(Split_RDD.map(lambda x: ([x[1],x[0],1/bc_djMap.value[x[0]]])).map(lambda x :x[1]).distinct())
N1 = sc.parallelize(range(Nodes)).map(lambda x: (x,init_value))
M_renormalize_part = M_no_values.cartesian(N1).map(lambda x: ([x[1][0],x[0],x[1][1]]))
M_new = M.union(M_renormalize_part)

## Build $r$

設定初始值r0，本次題目給定要求是$[\frac{beta}{N}]$,所以拿前面已經算過的來用

In [53]:
r0 = N1

### 跑第一次迴圈

$r_{old}$

每次代入的變數$r_{old}$，一開始是從r0開始

In [54]:
r_old = r0

###  $\beta$*M

因為 $\beta$*Ｍ在每次iterate皆相同<br>
故可以先算完後存起來


In [55]:
betaM = M_new.map(lambda x: ([x[0],x[1],x[2]*beta]))
betaM.persist()
betaM.is_cached

True

### Random page: (1- $\beta$)*1/N

In [56]:
def randomPage(x):
    return (x[0],x[1]+(1-beta)*init_value)

### $\beta$* M* $r_{old}$

和上次作業矩陣乘法的方法類似  

In [57]:
beta_Mr = betaM.map(lambda x: (x[1],(x[0],x[2]))).join(r_old).map(lambda x: (x[1][0][0],x[1][0][1]*x[1][1])).reduceByKey(lambda x,y: x+y)

$r_{new} = \beta M\cdot r_{old} + [\frac {1-\beta} {N}]_N$

In [58]:
r_new = beta_Mr.map(randomPage)

完成第一個iterate

### 印出前10高的page

* top(n): 取得前n個key值

In [59]:
#spec example result
#r_new.map(lambda x: (x[1],x[0])).top(5)

In [60]:
#real result
r_new.map(lambda x: (x[1],x[0])).top(10)

[(0.000618458508059618, 1054),
 (0.0005633413973307743, 1056),
 (0.0005455351388774083, 1536),
 (0.00048740654687620937, 407),
 (0.00046056000840176555, 261),
 (0.0004593344055583661, 410),
 (0.000457068897272089, 1198),
 (0.00045338147746186517, 4054),
 (0.00045075518565457836, 453),
 (0.0004376820886583324, 127)]

### Iteration

**run (start, times, $r_{old}$)**
* start: 從哪次開始
* times: 跑幾次迴圈
* $r_{old}$: 一開始代入的$r_{old}$ <br>

反覆call run並執行

In [37]:
def run(start,times,r_old):
    for i in range(times):
        beta_Mr = betaM.map(lambda x: (x[1],(x[0],x[2])))\
                .join(r_old).map(lambda x: (x[1][0][0],x[1][0][1]*x[1][1])).reduceByKey(lambda x,y: x+y)
        r_new = beta_Mr.map(randomPage)
        print("Iteration %d :" % (start+i) + str(r_new.map(lambda x: (x[1],x[0])).top(20)))
        r_old = r_new
    return r_new

下面使用spec中的example執行,以確定做法無誤

In [38]:
#r10 = run(1,10,r0)

Iteration 1 :[(0.3653333333333334, 3), (0.20533333333333337, 0), (0.15200000000000002, 2), (0.15200000000000002, 1), (0.12533333333333335, 4)]
Iteration 2 :[(0.31072000000000005, 3), (0.18912, 0), (0.18058666666666667, 2), (0.18058666666666667, 1), (0.1389866666666667, 4)]
Iteration 3 :[(0.33793564444444446, 3), (0.19346631111111112, 0), (0.1653632, 2), (0.1653632, 1), (0.13787164444444444, 4)]
Iteration 4 :[(0.3256057742222222, 3), (0.1933152142222222, 0), (0.17145622755555556, 2), (0.17145622755555556, 1), (0.13816655644444445, 4)]
Iteration 5 :[(0.33025018917925925, 3), (0.1930852071348148, 0), (0.16942300956444445, 2), (0.16942300956444445, 1), (0.13781858455703705, 4)]
Iteration 6 :[(0.32868534096023705, 3), (0.1931469333086815, 0), (0.1700741131226074, 2), (0.1700741131226074, 1), (0.13801949948586667, 4)]
Iteration 7 :[(0.3292098416787658, 3), (0.19315055118067992, 0), (0.16984842787711052, 2), (0.16984842787711052, 1), (0.13794275138633322, 4)]
Iteration 8 :[(0.329022331625387,

In [40]:
#Ans = r10.map(lambda x: (x[1],x[0]))
#output = Ans.top(10)
#for item in output:
#    item = (item[1],item[0])
#    item = str(item)
#    item = item.replace("(","").replace(")","").replace(", ","\t")
#    print (item)

3	0.32906630475610793
0	0.1931454977244748
2	0.16991320025033024
1	0.16991320025033024
4	0.13796179701875677


下面為使用p2p-Gnutella04.txt執行後的結果

Iteration 第1~5次

In [20]:
r5 = run(1,5,r0)

Iteration 1 :[(0.0006184585080596142, 1054), (0.0005633413973307736, 1056), (0.0005455351388774042, 1536), (0.0004874065468762122, 407), (0.0004605600084017689, 261), (0.000459334405558372, 410), (0.00045706889727208566, 1198), (0.0004533814774618591, 4054), (0.0004507551856545739, 453), (0.000437682088658336, 127), (0.0004368650200960558, 171), (0.00043418036624861785, 165), (0.00042379192309981455, 263), (0.00040438654474600206, 345), (0.00040418227760543495, 982), (0.00040411942617756523, 987), (0.00039763113859727, 763), (0.000390292112046925, 2265), (0.00039029211204692294, 5315), (0.00038375556354879547, 989)]
Iteration 2 :[(0.0006325012736368647, 1054), (0.0006249002957578554, 1056), (0.0005226566965516319, 1536), (0.0004993855291876081, 171), (0.0004927087708543705, 453), (0.00047522679681873375, 407), (0.0004712047477824946, 263), (0.00046377675661609925, 1959), (0.00046102686091041795, 165), (0.00046092221149465274, 261), (0.0004597552565281869, 410), (0.00045933667692517105,

Iteration 第6~10次

In [21]:
r10 = run(6,5,r5)

Iteration 6 :[(0.0006320859564577303, 1056), (0.000629059965147617, 1054), (0.0005238070755912338, 1536), (0.0005115756050901461, 171), (0.0004956152891154164, 453), (0.0004847393502147824, 407), (0.000479490910274371, 263), (0.0004703229444167316, 4664), (0.00046281216644022375, 261), (0.0004614306282047907, 410), (0.0004604336721452394, 1959), (0.00045966103157753257, 165), (0.0004404931723839469, 1198), (0.00042829341218305955, 127), (0.00041940270666811935, 4054), (0.00041047063902783306, 2265), (0.0004103271155468588, 345), (0.0004090889829877609, 763), (0.000400343233071042, 989), (0.00039940681448927403, 987)]
Iteration 7 :[(0.0006320893035982622, 1056), (0.0006290424725374235, 1054), (0.0005238070420505878, 1536), (0.0005115393839020572, 171), (0.000495582309056819, 453), (0.0004847616336196345, 407), (0.0004795306631455663, 263), (0.000470428469523849, 4664), (0.00046281425622855376, 261), (0.00046143366561834335, 410), (0.00046044263216706905, 1959), (0.000459650029718828, 16

Iteration 第11~15次

In [22]:
r15 = run(11,5,r10)

Iteration 11 :[(0.0006320902774181867, 1056), (0.0006290476156196339, 1054), (0.0005238203278753979, 1536), (0.0005115346105320632, 171), (0.0004955735406825305, 453), (0.0004847609925847326, 407), (0.00047953701210587884, 263), (0.00047041672885213086, 4664), (0.0004628120996654568, 261), (0.0004614308334365331, 410), (0.00046045000320310743, 1959), (0.0004596487191060733, 165), (0.0004404429051866898, 1198), (0.00042827581389280297, 127), (0.0004194359861013035, 4054), (0.00041045935062826336, 2265), (0.00041029088722494315, 345), (0.0004090694032803159, 763), (0.0004003388022777245, 989), (0.00039941047498430113, 987)]
Iteration 12 :[(0.0006320902544346259, 1056), (0.000629047676364164, 1054), (0.0005238203751437119, 1536), (0.0005115346264321643, 171), (0.0004955735346258258, 453), (0.0004847609595389547, 407), (0.0004795369466505335, 263), (0.00047041679225897737, 4664), (0.00046281210444254646, 261), (0.0004614307962730594, 410), (0.0004604500047477837, 1959), (0.0004596487375744

Iteration 第16~20次

In [23]:
r20 = run(16,5,r15)

Iteration 16 :[(0.0006320902603727955, 1056), (0.000629047686227607, 1054), (0.000523820383783535, 1536), (0.0005115346245252021, 171), (0.0004955735425191605, 453), (0.00048476095137786645, 407), (0.00047953693816532204, 263), (0.00047041676631967685, 4664), (0.00046281210761649617, 261), (0.0004614307965555513, 410), (0.00046045001722898554, 1959), (0.0004596487316903083, 165), (0.00044044294126014435, 1198), (0.00042827580643154414, 127), (0.000419436029818528, 4054), (0.00041045938279459133, 2265), (0.0004102908763668258, 345), (0.0004090694207608903, 763), (0.0004003388391775627, 989), (0.00039941047611768954, 987)]
Iteration 17 :[(0.0006320902603989738, 1056), (0.000629047686185539, 1054), (0.000523820383811346, 1536), (0.0005115346245078247, 171), (0.0004955735425609579, 453), (0.00048476095139063987, 407), (0.0004795369381921764, 263), (0.0004704167664742776, 4664), (0.0004628121076182037, 261), (0.00046143079654719244, 410), (0.00046045001718368085, 1959), (0.00045964873168873

把完成20次迴圈的答案格式做處理

In [28]:
Ans = r20.map(lambda x: (x[1],x[0]))

In [29]:
output = Ans.top(10)
for item in output:
    item = (item[1],item[0])
    item = str(item)
    item = item.replace("(","").replace(")","").replace(", ","\t")
    print (item)

1056	0.0006320902604125112
1054	0.0006290476861855152
1536	0.0005238203838152769
171	0.0005115346245042202
453	0.000495573542576762
407	0.00048476095139752943
263	0.00047953693819939504
4664	0.0004704167665000125
261	0.00046281210761336424
410	0.0004614307965481321


### **Ans**
Top10 <br><br>
1056    0.000632<br>
1054    0.000629<br>
1536    0.000523<br>
171 0.000511<br>
453 0.000495<br>
407 0.000484<br>
263 0.000479<br>
4664    0.000470<br>
261 0.000462<br>
410 0.000461<br>